### Plan

1. **GitHubPlugin**:
   - Fetch repository contents using GitHub API
   - Extract code files with specific extensions
   - Parse directory structure

2. **CodeAnalysisPlugin**:
   - Identify functions and classes
   - Extract comments and docstrings
   - Analyze code complexity
   - Detect dependencies

3. **Semantic Memory**:
   - Store analyzed code segments
   - Remember relationships between files
   - Track dependencies
   - Store generated documentation drafts

4. **Semantic Functions**:
   - Explain code functionality
   - Suggest improvements
   - Create usage examples
   - Write test cases

5. **Stepwise Planner**:
   - Plan the documentation structure
   - Prioritize important components
   - Generate documentation in chunks
   - Create a final markdown document

### Implementation

#### 1. GitHubPlugin



In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

# Set up kernel with OpenAI
kernel = Kernel()
service_id = "default"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
    ),
)

In [2]:
import requests
from typing import List, Dict

class GitHubPlugin:
    def __init__(self):
        self.headers = {
            "Accept": "application/vnd.github.v3+json"
        }

    def fetch_repo_contents(self, owner: str, repo: str, path: str = "") -> List[Dict]:
        url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
        response = requests.get(url, headers=self.headers)
        response.raise_for_status()
        return response.json()

    def extract_code_files(self, contents: List[Dict], extensions: List[str]) -> List[Dict]:
        return [item for item in contents if any(item['name'].endswith(ext) for ext in extensions)]

    def parse_directory_structure(self, contents: List[Dict]) -> Dict:
        structure = {}
        for item in contents:
            parts = item['path'].split('/')
            current = structure
            for part in parts[:-1]:
                current = current.setdefault(part, {})
            current[parts[-1]] = item['type']
        return structure



#### 2. CodeAnalysisPlugin



In [3]:
import ast
from typing import List, Dict

class CodeAnalysisPlugin:
    def analyze_code(self, code: str) -> Dict:
        tree = ast.parse(code)
        return {
            "functions": self._get_functions(tree),
            "classes": self._get_classes(tree),
            "comments": self._get_comments(code),
            "complexity": self._get_complexity(tree),
            "dependencies": self._get_dependencies(tree)
        }

    def _get_functions(self, tree: ast.AST) -> List[str]:
        return [node.name for node in ast.walk(tree) if isinstance(node, ast.FunctionDef)]

    def _get_classes(self, tree: ast.AST) -> List[str]:
        return [node.name for node in ast.walk(tree) if isinstance(node, ast.ClassDef)]

    def _get_comments(self, code: str) -> List[str]:
        return [line.strip() for line in code.split('\n') if line.strip().startswith('#')]

    def _get_complexity(self, tree: ast.AST) -> int:
        return sum(1 for node in ast.walk(tree) if isinstance(node, (ast.If, ast.For, ast.While)))

    def _get_dependencies(self, tree: ast.AST) -> List[str]:
        return [node.module for node in ast.walk(tree) if isinstance(node, ast.Import)]



#### 3. Semantic Memory



In [4]:
from semantic_kernel.memory import VolatileMemoryStore
from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

embedding_service_id = "embeddings"
memory_store = VolatileMemoryStore()
embeddings = AzureTextEmbedding(service_id=embedding_service_id)
memory = SemanticTextMemory(memory_store, embeddings)

async def store_code_segment(memory, collection, id, text, description, metadata):
    await memory.save_information(
        collection=collection,
        id=id,
        text=text,
        description=description,
        additional_metadata=metadata
    )



#### 4. Semantic Functions



In [5]:
from semantic_kernel.functions import kernel_function, KernelFunctionFromPrompt

explain_function = KernelFunctionFromPrompt(
    function_name="ExplainCode",
    plugin_name="CodeAnalysisPlugin",
    prompt="Explain the functionality of the following code: {{$input}}",
    description="Explains code functionality."
)

suggest_improvements_function = KernelFunctionFromPrompt(
    function_name="SuggestImprovements",
    plugin_name="CodeAnalysisPlugin",
    prompt="Suggest improvements for the following code: {{$input}}",
    description="Suggests improvements for the code."
)

create_examples_function = KernelFunctionFromPrompt(
    function_name="CreateExamples",
    plugin_name="CodeAnalysisPlugin",
    prompt="Create usage examples for the following code: {{$input}}",
    description="Creates usage examples for the code."
)

write_tests_function = KernelFunctionFromPrompt(
    function_name="WriteTests",
    plugin_name="CodeAnalysisPlugin",
    prompt="Write test cases for the following code: {{$input}}",
    description="Writes test cases for the code."
)

# Add functions to the kernel
kernel.add_function(plugin_name="CodeAnalysisPlugin", function=explain_function)
kernel.add_function(plugin_name="CodeAnalysisPlugin", function=suggest_improvements_function)
kernel.add_function(plugin_name="CodeAnalysisPlugin", function=create_examples_function)
kernel.add_function(plugin_name="CodeAnalysisPlugin", function=write_tests_function)

KernelFunctionFromPrompt(metadata=KernelFunctionMetadata(name='WriteTests', plugin_name='CodeAnalysisPlugin', description='Writes test cases for the code.', parameters=[KernelParameterMetadata(name='input', description='', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object'}, include_in_function_choices=True)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True), additional_properties=None), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x7f0938666900>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x7f0938666990>, prompt_template=KernelPromptTemplate(prompt_template_config=PromptTemplateConfig(name='WriteTests', description='Writes



#### 5. Stepwise Planner



In [6]:
from semantic_kernel.planners import FunctionCallingStepwisePlanner, FunctionCallingStepwisePlannerOptions

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000,
)

planner = FunctionCallingStepwisePlanner(service_id="default", options=options)

async def plan_documentation(kernel, repo_owner, repo_name):
    task = f"""
    Plan the documentation structure for the repository {repo_owner}/{repo_name}.
    Prioritize important components.
    Generate documentation in chunks.
    Create a final markdown document.
    """
    result = await planner.invoke(kernel, task)
    return result.final_answer

In [7]:
# Example usage of GitHubPlugin
github_plugin = GitHubPlugin()
repo_contents = github_plugin.fetch_repo_contents(owner="RealCloudLike", repo="wd-ai-hackathon")
code_files = github_plugin.extract_code_files(contents=repo_contents, extensions=[".py", ".js"])
directory_structure = github_plugin.parse_directory_structure(contents=repo_contents)
print(directory_structure)

{'.devcontainer': 'dir', '.dockerignore': 'file', '.env_template': 'file', '.gitignore': 'file', '.python-version': 'file', 'LICENSE': 'file', 'README.md': 'file', 'notebooks': 'dir', 'pyproject.toml': 'file', 'semantic-kernel': 'dir', 'uv.lock': 'file'}


In [8]:
# Example usage of CodeAnalysisPlugin
code_analysis_plugin = CodeAnalysisPlugin()
sample_code = """
def example_function():
    # This is a sample function
    print("Hello, world!")
"""
analysis_result = code_analysis_plugin.analyze_code(sample_code)
print(analysis_result)

{'functions': ['example_function'], 'classes': [], 'comments': ['# This is a sample function'], 'complexity': 0, 'dependencies': []}


In [9]:
# Example usage of Semantic Memory
await store_code_segment(
    memory=memory,
    collection="code_segments",
    id="example_function",
    text=sample_code,
    description="Example function for demonstration",
    metadata={"author": "user"}
)

In [11]:
from semantic_kernel.functions import KernelArguments

# Example usage of Semantic Functions
async def use_semantic_functions():
	explain_result = await kernel.invoke(explain_function, KernelArguments(input=sample_code))
	print(explain_result)

	suggest_result = await kernel.invoke(suggest_improvements_function, KernelArguments(input=sample_code))
	print(suggest_result)

	examples_result = await kernel.invoke(create_examples_function, KernelArguments(input=sample_code))
	print(examples_result)

	tests_result = await kernel.invoke(write_tests_function, KernelArguments(input=sample_code))
	print(tests_result)

await use_semantic_functions()

The code you provided defines a simple Python function called `example_function`. Here’s a breakdown of its functionality:

1. **Function Definition**: 
   - The line `def example_function():` defines a function named `example_function`. In Python, `def` is a keyword used to declare a function.
   
2. **Docstring/Comment**: 
   - The line `# This is a sample function` is a comment (signified by the `#` symbol). Comments are not executed as part of the program; they are meant for documentation purposes to explain what the code does. In this case, it indicates that the function is a sample or example.

3. **Print Statement**: 
   - The line `print("Hello, world!")` is an instruction that will output the string "Hello, world!" to the console when the function is called. The `print()` function is a built-in Python function used to display information to the standard output stream.

To execute this function and see the output, you would need to call it by adding `example_function()` after i

In [13]:
# Example usage of Stepwise Planner
documentation_plan = await plan_documentation(kernel, repo_owner="RealCloudLike", repo_name="wd-ai-hackathon")
print(documentation_plan)

The next step in the plan is to create documentation chunks for each identified component. Here are the proposed documentation chunks:

1. **Overview**  
    - Brief description of the project  
    - Purpose of the project  
    - Key features  

2. **Installation**  
    - System requirements  
    - Step-by-step installation instructions  
    - Common installation issues and troubleshooting tips  

3. **Usage**  
    - Basic usage example  
    - Advanced usage scenarios  
    - FAQs  

4. **API Reference**  
    - Overview of API structure  
    - List of available endpoints  
    - Sample requests and responses  
    - Authentication methods

5. **Contributing**  
    - Guidelines for contributing  
    - Code of conduct  
    - Steps for submitting pull requests  

6. **License**  
    - Licensing information  
    - Contribution under the license

Would you like to proceed with creating these documentation chunks in detail?
